In [72]:
import numpy as np 
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime

import time
import matplotlib.pyplot as plt

train_origin = pd.read_csv("train.csv")
train_origin.head()


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [73]:
train = train_origin.drop_duplicates(['Dates', 'Category', 'Descript','Address'], keep='first')
# train = train_origin.drop_duplicates(['Dates', 'Category', 'Descript'], keep='first')



In [74]:
np.shape(train_origin)

(878049, 9)

In [75]:
np.shape(train)

(875357, 9)

In [76]:
train['Time'] = train['Dates'].apply(lambda x: int(x[10:13]))
train.set_index('Time')
train['Time'].value_counts()

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


18    54961
17    53385
12    51770
16    50006
19    49336
15    47902
22    45590
0     44666
20    44561
14    44311
21    43515
13    43044
23    42340
11    38258
10    37684
9     35424
8     32759
1     26118
2     22257
7     21949
3     13986
6     13083
4      9844
5      8608
Name: Time, dtype: int64

In [77]:
ord_enc = OrdinalEncoder(dtype=np.int)
col_trans = ColumnTransformer([('ord_enc', ord_enc, ['Category', 'DayOfWeek', 'PdDistrict','Address'])])
data = col_trans.fit_transform(train)
data

array([[   37,     6,     4, 19790],
       [   21,     6,     4, 19790],
       [   21,     6,     4, 22697],
       ...,
       [   16,     1,     7, 11315],
       [   35,     1,     7, 22308],
       [   12,     1,     0,  5128]])

In [78]:
X = pd.DataFrame(data=data, columns=['Category', 'DayOfWeek', 'PdDistrict', 'Address'])

X['Time'] = train['Dates'].apply(lambda x: int(x[10:13]))
# X['TimeStamp'] = train['Dates'].apply(lambda x: time.mktime(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timetuple()))
X['X'] = train['X']
X['Y'] = train['Y']
# X['Count']
X.head()
print(np.shape(X))

(875357, 7)


In [79]:
X.head()

,Category,DayOfWeek,PdDistrict,Address,Time,X,Y
0,37,6,4,19790,23.0,-122.425892,37.774599
1,21,6,4,19790,23.0,-122.425892,37.774599
2,21,6,4,22697,23.0,-122.424363,37.800414
3,16,6,4,4266,23.0,-122.426995,37.800873
4,16,6,5,1843,23.0,-122.438738,37.771541


In [80]:
X = X.dropna(axis=0)
X.head()

,Category,DayOfWeek,PdDistrict,Address,Time,X,Y
0,37,6,4,19790,23.0,-122.425892,37.774599
1,21,6,4,19790,23.0,-122.425892,37.774599
2,21,6,4,22697,23.0,-122.424363,37.800414
3,16,6,4,4266,23.0,-122.426995,37.800873
4,16,6,5,1843,23.0,-122.438738,37.771541


In [81]:
print(np.shape(X))

(872677, 7)


In [82]:
import math

def ScaleConverter(X):
    x_mean = np.mean(X.values)
    x_std = math.sqrt(np.var(X.values))
    X = X.apply(lambda x: (x - x_mean) / x_std)
    return X



X['DayOfWeek'] = X['DayOfWeek'] / 6
X['PdDistrict'] = X['PdDistrict'] / 9
X['X'] = ScaleConverter(X['X'])
X['Y'] = ScaleConverter(X['Y'])
# X['Time'] = ScaleConverter(X['Time'])
X['Address'] = X['Address'] / np.max(X['Address'])
X.head()

,Category,DayOfWeek,PdDistrict,Address,Time,X,Y
0,37,1.0,0.444444,0.852026,23.0,-0.107471,0.007735
1,21,1.0,0.444444,0.852026,23.0,-0.107471,0.007735
2,21,1.0,0.444444,0.977182,23.0,-0.057157,0.064065
3,16,1.0,0.444444,0.183666,23.0,-0.143796,0.065065
4,16,1.0,0.555556,0.079347,23.0,-0.530282,0.001063


In [83]:
X['Time'] = X['Time'].map({
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 3,
    10: 3,
    11: 4,
    12: 4,
    13: 4,
    14: 4,
    15: 4,
    16: 4,
    17: 5,
    18: 5,
    19: 5,
    20: 5,
    21: 5,
    22: 6,
    23: 6
})

In [84]:
X['Time'] = X['Time'] / 6

In [85]:
X.head()

,Category,DayOfWeek,PdDistrict,Address,Time,X,Y
0,37,1.0,0.444444,0.852026,1.0,-0.107471,0.007735
1,21,1.0,0.444444,0.852026,1.0,-0.107471,0.007735
2,21,1.0,0.444444,0.977182,1.0,-0.057157,0.064065
3,16,1.0,0.444444,0.183666,1.0,-0.143796,0.065065
4,16,1.0,0.555556,0.079347,1.0,-0.530282,0.001063


In [86]:
from sklearn.model_selection import train_test_split

train_data = X.drop('Category', axis=1)
# target_data = np.array(X['Category'])

x_train, x_test, y_train, y_test  = train_test_split(train_data, target_data)

print(train_data.shape, x_train.shape, x_test.shape)

(872677, 6) (654507, 6) (218170, 6)


In [87]:
target_data

array([37, 21, 21, ..., 16, 35, 12])

In [88]:
x_train

,DayOfWeek,PdDistrict,Address,Time,X,Y
846547,0.333333,1.000000,0.018125,0.666667,0.983209,-0.020682
33252,0.333333,0.777778,0.008482,0.166667,0.410417,0.028897
284586,0.333333,0.444444,0.524476,0.666667,0.942748,0.052383
702957,1.000000,0.555556,0.571447,0.666667,0.524250,0.026030
578066,0.500000,0.000000,0.408404,0.833333,0.604179,0.029293
...,...,...,...,...,...,...
142707,0.166667,0.777778,0.529901,0.833333,-0.204605,-0.087789
92731,0.833333,0.333333,0.658587,0.666667,-0.088159,-0.038451
449217,0.666667,0.111111,0.468463,0.833333,0.143318,-0.019968
237290,0.166667,0.000000,0.172256,1.000000,0.145536,0.046414


In [89]:
y_train

array([37,  7, 38, ..., 16, 20, 21])

In [90]:
x_test

,DayOfWeek,PdDistrict,Address,Time,X,Y
741767,0.333333,1.000000,0.066819,0.666667,0.465010,0.061109
718258,0.000000,0.777778,0.529901,0.833333,0.256115,-0.023029
163450,0.500000,0.111111,0.477505,0.833333,0.530507,0.020243
709511,0.500000,0.000000,0.077022,0.166667,0.060884,-0.086342
788474,0.666667,1.000000,0.103156,0.500000,-1.463874,0.025566
...,...,...,...,...,...,...
188449,0.500000,1.000000,0.470961,0.833333,0.029383,-0.016920
746380,0.166667,0.333333,0.284970,0.666667,0.192271,-0.016138
230798,0.000000,0.333333,0.394412,1.000000,1.044374,-0.085194
724320,0.666667,0.000000,0.047574,1.000000,-0.221267,-0.049516


In [91]:
y_train[:]

array([37,  7, 38, ..., 16, 20, 21])

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
# forest = RandomForestClassifier(n_estimators=10, max_depth=None)
# forest = RandomForestClassifier(n_estimators=10, max_depth=20,min_samples_split=2, random_state=0)
# forest.fit(x_train, y_train)

forest = RandomForestClassifier(n_estimators=100, max_depth=30,min_samples_split=2, random_state=0,min_samples_leaf=2)
forest.fit(x_train, y_train)
y_train_pred = forest.predict(x_train)
y_test_pred = forest.predict(x_test)
print('Accuracy: %.2f' % accuracy_score(y_train, y_train_pred))
print('Accuracy: %.2f' % accuracy_score(y_test_pred, y_test))

Accuracy: 0.90
Accuracy: 0.24


In [48]:
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler


rbf_feature = RBFSampler(gamma=1, random_state=1,n_components=100)
X_features_train = rbf_feature.fit_transform(x_train)
X_features_test = rbf_feature.fit_transform(x_test)

clf = SGDClassifier(max_iter=100)
clf.fit(X_features_train, y_train)
y_train_pred = clf.predict(X_features_train)
y_test_pred = clf.predict(X_features_test)
print('Accuracy: %.2f' % accuracy_score(y_train, y_train_pred))
print('Accuracy: %.2f' % accuracy_score(y_test_pred, y_test))

Accuracy: 0.10
Accuracy: 0.09


In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=0)
svm.fit(x_train, y_train)
y_train_pred = svm.predict(x_train)
y_test_pred = svm.predict(x_test)

print('Accuracy: %.2f' % accuracy_score(y_train, y_train_pred))
print('Accuracy: %.2f' % accuracy_score(y_test_pred, y_test))

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1.0, random_state=0)
svm.fit(x_train, y_train)
y_train_pred = svm.predict(x_train)
y_test_pred = svm.predict(x_test)

print('Accuracy: %.2f' % accuracy_score(y_train, y_train_pred))
print('Accuracy: %.2f' % accuracy_score(y_test_pred, y_test))